# NLP Tweet Analyzer Kaggle Submission

In [2]:
import os
import sys
import pandas as pd
import numpy as np
from tqdm import tqdm
import gensim
import gensim.downloader as api
from gensim.models import KeyedVectors

# NLP Preprocessing
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# NLP Viz
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# Data Modelling
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, recall_score
from sklearn.decomposition import PCA, TruncatedSVD
from sklearn.manifold import TSNE
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
import xgboost
from xgboost import XGBClassifier
xgboost.config_context(verbosity=0) # Silect XGBoost

# Monitoring progress
from tqdm.notebook import tqdm_notebook
tqdm_notebook.pandas()

nltk.download('stopwords')

config = {
    'test_size': 0.2,
    'CV_splits': 5,
    'seed': 14,
    'n_cores': 16,
    'max_features': 10000,
    'refresh_embedding': False
}

/Users/miguelcachosoblechero/opt/anaconda3/envs/Kaggle/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/miguelcachosoblechero/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Generate Embeddings
def average_embedding(tweet, word_embeddings, missing_random=False, dimension=300):
    
    # Create target embedding
    final_emb = np.zeros(dimension)
    num_words = len(tweet)

    # For each word in sentence
    for word in tweet:

        # Look for key
        if word in word_embeddings:
            final_emb += word_embeddings[word]

        # If missing...
        else:
            if missing_random:
                final_emb += np.random.rand(dimension)
            else:
                final_emb += np.zeros(dimension)
                
    # Average embeddings
    if num_words == 0:
        return np.zeros(dimension)
    else:
        return final_emb/num_words

In [4]:
# 1- Preprocessing and Feature Engineering
# Load dataset
input_path = "../input"
raw_tweets_train = pd.read_csv(os.path.join(input_path, "train.csv")).drop(['id'], axis=1)
raw_tweets_test = pd.read_csv(os.path.join(input_path, "test.csv"))

# Extract data and labels
X_train = raw_tweets_train.drop(['target'], axis=1)
y_train = raw_tweets_train.target.values
X_test = raw_tweets_test

# Use Twitter Tokenizer to tokenize tweets
tokenizer = TweetTokenizer()
X_train['twitterTokens'] = X_train.apply(lambda x: tokenizer.tokenize(x.text.lower()), axis=1)
X_test['twitterTokens'] = X_test.apply(lambda x: tokenizer.tokenize(x.text.lower()), axis=1)

# Remove stop words
english_stopwords = set(stopwords.words('english'))
X_train['twitterTokens_noStop'] = X_train.twitterTokens.apply(lambda x: [i for i in x if i not in english_stopwords])
X_test['twitterTokens_noStop'] = X_test.twitterTokens.apply(lambda x: [i for i in x if i not in english_stopwords])

# Tokenize + Stop Words + BoW
CountVec = CountVectorizer(stop_words='english', max_features=config['max_features'])
X_train_bow = CountVec.fit_transform(X_train.text)
X_test_bow = CountVec.transform(X_test.text)

# TfidfVectorizer can be used to perform this action in normal text
CountVecTFIDF = TfidfVectorizer(stop_words='english', max_features=config['max_features'])
X_train_tfidf = CountVecTFIDF.fit_transform(X_train.text)
X_test_tfidf = CountVecTFIDF.transform(X_test.text)

# Download or load from local
if config['refresh_embedding']:
    # Download Google's pre-trained Word2Vec model
    word2vec = api.load('word2vec-google-news-300')
    # Save the model for future reuse
    word2vec.save_word2vec_format('../Word2Vec/word2vec_300.kv')
else:
    word2vec = KeyedVectors.load_word2vec_format('../Word2Vec/word2vec_300.kv')

# Create embeddings by averaging the sentence
X_train_embeddings = pd.DataFrame(X_train.twitterTokens_noStop.apply(average_embedding, word_embeddings=word2vec, missing_random=False).tolist())
X_test_embeddings = pd.DataFrame(X_test.twitterTokens_noStop.apply(average_embedding, word_embeddings=word2vec, missing_random=False).tolist())

In [5]:
# 2- Data modelling
# Select your model
target_model = XGBClassifier(n_estimators=20, max_depth=50, random_state=config['seed'], n_jobs = config['n_cores'])
# target_model = MultinomialNB()

# Train your model
target_model.fit(X_train_embeddings, y_train)

# Generate predictions
results = target_model.predict(X_test_embeddings)

# Store results
pd.DataFrame({"id": raw_tweets_test.id,
              "target": results}).set_index("id").to_csv("../submission/nlp_submission.csv")

/Users/miguelcachosoblechero/opt/anaconda3/envs/Kaggle/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/Users/miguelcachosoblechero/opt/anaconda3/envs/Kaggle/lib/python3.8/site-packages/xgboost/data.py:250: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  elif isinstance(data.columns, (pd.Int64Index, pd.RangeIndex)):


[22:43:26] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
